In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/'My Drive'/'kaggle'/'LinkingWritingProcess'

/content/drive/My Drive/kaggle/LinkingWritingProcess


In [14]:
!pip install catboost

In [15]:
!pip install optuna

## (1) Imports

In [16]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

import re
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import datetime as dt
from tqdm.auto import tqdm
tqdm.pandas()

from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.ensemble import AdaBoostRegressor

from scipy.optimize import minimize

./data/sample_submission.csv
./data/test_logs.csv
./data/train_logs.csv
./data/train_scores.csv
./data/Essay Constructor Result.csv


## (2) Load data

In [17]:
# 訓練データ読み込み
train_logs = pd.read_csv("./data/train_logs.csv")
train_scores = pd.read_csv("./data/train_scores.csv")
# 検証データ読み込み
test_logs = pd.read_csv("./data/test_logs.csv")

print("train_df： ", train_logs.shape)
print("train_scores: ", train_scores.shape)
print("test_df: ", test_logs.shape)

train_df：  (8405898, 11)
train_scores:  (2471, 2)
test_df:  (6, 11)


## (3) Helper functions

In [18]:
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',',
          'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def activity_counts(df):
    tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['activity'].values):
        items = list(Counter(li).items())
        di = dict()
        for k in activities:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


def event_counts(df, colname):
    tmp_df = df.groupby('id').agg({colname: list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df[colname].values):
        items = list(Counter(li).items())
        di = dict()
        for k in events:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


def text_change_counts(df):
    tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['text_change'].values):
        items = list(Counter(li).items())
        di = dict()
        for k in text_changes:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
def match_punctuations(df):
    tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['down_event'].values):
        cnt = 0
        items = list(Counter(li).items())
        for item in items:
            k, v = item[0], item[1]
            if k in punctuations:
                cnt += v
        ret.append(cnt)
    ret = pd.DataFrame({'punct_cnt': ret})
    return ret


def get_input_words(df):
    tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
    tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
    tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
    tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
    tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
    tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df.drop(['text_change'], axis=1, inplace=True)
    return tmp_df

## (4) Make features

In [19]:
def make_feats(df):

    # id
    feats = pd.DataFrame({'id': df['id'].unique().tolist()})

    # time shift
    df['up_time_shift1'] = df.groupby('id')['up_time'].shift(1)
    df['action_time_gap'] = df['down_time'] - df['up_time_shift1']
    df.drop('up_time_shift1', axis=1, inplace=True)

    # cursor position shift
    df['cursor_position_shift1'] = df.groupby('id')['cursor_position'].shift(1)
    df['cursor_position_change'] = np.abs(df['cursor_position'] - df['cursor_position_shift1'])
    df.drop('cursor_position_shift1', axis=1, inplace=True)

    # word count shift
    df['word_count_shift1'] = df.groupby('id')['word_count'].shift(1)
    df['word_count_change'] = np.abs(df['word_count'] - df['word_count_shift1'])
    df.drop('word_count_shift1', axis=1, inplace=True)

    # stats feats
    for item in tqdm([
        ('event_id', ['max']),
        ('up_time', ['max']),
        ('action_time', ['sum', 'max', 'mean', 'std']),
        ('activity', ['nunique']),
        ('down_event', ['nunique']),
        ('up_event', ['nunique']),
        ('text_change', ['nunique']),
        ('cursor_position', ['nunique', 'max', 'mean']),
        ('word_count', ['nunique', 'max', 'mean']),
        ('action_time_gap', ['max', 'min', 'mean', 'std', 'sum']),
        ('cursor_position_change', ['max', 'mean', 'std', 'sum']),
        ('word_count_change', ['max', 'mean', 'std', 'sum'])
    ]):
        colname, methods = item[0], item[1]
        for method in methods:
            tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method}'})
            feats = feats.merge(tmp_df, on='id', how='left')

    # counts
    tmp_df = activity_counts(df)
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = event_counts(df, 'down_event')
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = event_counts(df, 'up_event')
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = text_change_counts(df)
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = match_punctuations(df)
    feats = pd.concat([feats, tmp_df], axis=1)

    # input words
    tmp_df = get_input_words(df)
    feats = pd.merge(feats, tmp_df, on='id', how='left')

    # compare feats
    feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
    feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
    feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
    feats['idle_time_ratio'] = feats['action_time_gap_sum'] / feats['up_time_max']

    return feats

In [20]:
train_feats = make_feats(train_logs)
test_feats = make_feats(test_logs)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
train_feats = train_feats.merge(train_scores, on='id', how='left')
print('Train : ', train_feats.shape, ' Test : ', test_feats.shape)

Train :  (2471, 92)  Test :  (3, 91)


In [22]:
display(train_feats.head())
display(test_feats.head())

,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,...,punct_cnt,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std,word_time_ratio,word_event_ratio,event_time_ratio,idle_time_ratio,score
0,001519c8,2557,1801969,297243,2259,116.246774,91.797374,7,12,12,...,37,366,5.325137,20,3.487804,0.000142,0.100117,0.001419,0.832534,3.5
1,0022f953,2454,1788969,275391,1758,112.221271,55.431189,5,17,17,...,53,385,4.410390,33,3.199496,0.000181,0.131622,0.001372,0.828944,3.5
2,0042269b,4136,1771669,421201,3005,101.837766,82.383766,4,13,18,...,47,627,5.446571,25,3.474895,0.000228,0.097679,0.002335,0.759751,6.0
3,0059420b,1556,1404469,189596,806,121.848329,113.768226,5,15,15,...,18,251,4.609562,19,2.949601,0.000147,0.132391,0.001108,0.835531,2.0
4,0075873a,2531,1662472,313702,701,123.943896,62.082013,3,11,11,...,66,412,4.766990,18,2.986064,0.000152,0.099565,0.001522,0.764103,4.0


,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,...,text_change_14_count,punct_cnt,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std,word_time_ratio,word_event_ratio,event_time_ratio,idle_time_ratio
0,0000aaaa,2,760160,172,87,86.0,1.414214,1,1,1,...,0,0,0,0.0,0,0.0,0.000000,0.0,0.000003,0.554561
1,2222bbbb,2,712023,113,67,56.5,14.849242,1,1,1,...,0,0,1,2.0,2,0.0,0.000001,0.5,0.000003,-0.592005
2,4444cccc,2,635641,150,94,75.0,26.870058,1,2,2,...,0,0,1,1.0,1,0.0,0.000002,0.5,0.000003,-0.708962


## (5) Baseline

In [23]:
# 説明変数と目的変数を設定
X = train_feats.drop(columns = ['id', 'score'], axis=1)
y = train_feats.score
X_total = test_feats.drop(columns=['id'], axis=1)

print('train = ', X.shape, 'test = ', X_total.shape, 'target = ', y.shape)

train =  (2471, 90) test =  (3, 90) target =  (2471,)


In [24]:
# モデルの定義
model_1 = CatBoostRegressor(verbose=0)
model_2 = xgb.XGBRegressor()
model_3 = lgb.LGBMRegressor()
model_4 = RandomForestRegressor()
model_5 = HistGradientBoostingRegressor()

# クロスバリデーションを行なって精度の比較
print('Score baseline CatBoostRegressor: ', cross_val_score(model_1, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline XGBRegressor: ', cross_val_score(model_2, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline LGBMRegressor: ', cross_val_score(model_3, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline RandomForestRegressor: ', cross_val_score(model_4, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline HostGradientBoostingRegressor: ', cross_val_score(model_5, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())

Score baseline CatBoostRegressor:  -0.6579840647475601
Score baseline XGBRegressor:  -0.7087934576099262
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13741
[LightGBM] [Info] Number of data points in the train set: 1976, number of used features: 88
[LightGBM] [Info] Start training from score 3.710273
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13741
[LightGBM] [Info] Number of data points in the train set: 1977, number of used features: 88
[LightGBM] [Info] Start training from score 3.709914
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002208 seconds.
You can set `force_col_wise=true` to remove

## (6) Data preparetion

In [25]:
def variance_threshold(df, th):
  var_thres = VarianceThreshold(threshold=th)
  var_thres.fit(df)
  new_cols = var_thres.get_support()
  return df.iloc[:, new_cols]

In [26]:
X_fit = variance_threshold(X, 0.2)
list_name = (X_fit.columns)
X_total_fit = X_total[list_name]

print('Shape test = ', X_total_fit.shape)
print('Shape train = ', X_fit.shape)

Shape test =  (3, 85)
Shape train =  (2471, 85)


In [27]:
X_fit = X_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_total_fit = X_total_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print('Shape train = ', X_fit.shape)
print('Shape test = ', X_total_fit.shape)

Shape train =  (2471, 85)
Shape test =  (3, 85)


In [28]:
# 訓練データの標準化
std_ = StandardScaler()
X_std = std_.fit_transform(X_fit)
X_total_std = std_.fit_transform(X_total_fit)